[한국어 코퍼스 전처리]

- 데이터셋: Korpora에서 로딩
- 형태소 분석기 설정


- 단어 사전 생성 필요

---
[1] 모듈 로딩 및 데이터 준비

In [22]:
#모듈 로딩
from Korpora import Korpora                         #open Korean Dataset
from konlpy.tag import *                            #형태소 분석기
import spacy                                        #형태소 분석기

from torch.utils.data import Dataset, DataLoader    #Pytorch Dataset 관련모듈

import pandas as pd
import numpy as np
import re                                           #for 정규식
import string

In [23]:
#데이터 로딩
nsmc=Korpora.load('nsmc')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\MSI\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\MSI\K

In [24]:
nsmc.train[0]

LabeledSentence(text='아 더빙.. 진짜 짜증나네요 목소리', label=0)

In [25]:
nsmc.test[0]

LabeledSentence(text='굳 ㅋ', label=1)

In [26]:
#테스트 데이터셋 -> DF
nsmcDF=pd.DataFrame(nsmc.test)
nsmcDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


---
[2] 텍스트 데이터 전처리

- 토큰화
- 정제 작업(불용어, 구두점, 띄어쓰기, 오탈자 처리 등)

- 단어 사전 생성
- 문장 수치화

In [27]:
nsmcDF['text']=nsmcDF['text'].str.replace('\n', '')

In [28]:
#구두점 제거
punc=string.punctuation

for p in punc:
    nsmcDF['text'] = nsmcDF['text'].str.replace(p, '') 
    
nsmcDF

,text,label
0,굳 ㅋ,1
1,GDNTOPCLASSINTHECLUB,0
2,뭐야 이 평점들은 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,3D만 아니었어도 별 다섯 개 줬을텐데 왜 3D로 나와서 제 심기를 불편하게 하죠,0
...,...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,0
49998,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,0


In [29]:
#한국어 외 모든 문자 제거
m=re.compile('[^ ㄱ-ㅣ가-힣]+')

nsmcDF['text']=nsmcDF['text'].apply(lambda x: m.sub('', x))
nsmcDF

,text,label
0,굳 ㅋ,1
1,,0
2,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0
...,...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,0
49997,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,0
49998,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,0


In [9]:
#형태소 분석
kkma=Kkma()

In [14]:
for t in nsmcDF['text']:
    kkma.morphs(t)

KeyboardInterrupt: 

In [15]:
lang_model='ko_core_news_lg'

In [16]:
nlp=spacy.load(lang_model)

In [32]:
my_token=[]

for t in nsmcDF['text']:
    doc=nlp(t)

    for token in doc:
        if (not token.is_punct) or (not token.is_stop):
            my_token.append(token)

my_token

[굳,
 ㅋ,
 뭐야,
 이,
 평점들은,
 나쁘진,
 않지만,
 점,
 짜리는,
 더더욱,
 아니잖아,
 지루하지는,
 않은데,
 완전,
 막장임,
 돈주고,
 보기에는,
 만,
 아니었어도,
 별,
 다섯,
 개,
 줬을텐데,
 왜,
 로,
 나와서,
 제,
 심기를,
 불편하게,
 하죠,
 음악이,
 주가,
 된,
 최고의,
 음악영화,
 진정한,
 쓰레기,
 마치,
 미국애니에서,
 튀어나온듯한,
 창의력없는,
 로봇디자인부터가고개를,
 젖게한다,
 갈수록,
 개판되가는,
 중국영화,
 유치하고,
 내용없음,
 폼잡다,
 끝남,
 말도안되는,
 무기에,
 유치한남무,
 아,
 그립다,
 동사서독같은,
 영화가,
 이건,
 류아류작이다,
 이별의,
 아픔뒤에,
 찾아오는,
 새로운,
 인연의,
 기쁨,
  ,
 모든,
 사람이,
 그렇지는,
 않네,
 괜찮네요오랜만포켓몬스터잼밌어요,
 한국독립영화의,
 한계,
 그렇게,
 아버지가,
 된다와,
 비교됨,
 청춘은,
 아름답다,
 그,
 아름다움은,
 이성을,
 흔들어,
 놓는다,
 찰나의,
 아름다움을,
 잘,
 포착한,
 섬세하고,
 아름다운,
 수채화같은,
 퀴어영화이다,
 눈에,
 보이는,
 반전이었지만,
 영화의,
 흡인력은,
 사라지지,
 않았다,
 스토리,
 연출,
 연기,
 비주얼,
 등,
 영화의,
 기본,
 조차,
 안된,
 영화에,
 무슨,
 평을,
 해,
 이런,
 영화,
 찍고도,
 김문옥,
 감독은,
 내가,
 영화,
 경력이,
 몇인데,
 조무래기들이,
 내,
 영화를,
 평론해,
 같은,
 마인드에,
 빠져있겠지,
 소위,
 ㅈ문가라는,
 평점은,
 뭐냐,
 최고,
 발연기,
 도저히,
 못보겠다,
 진짜,
 이렇게,
 연기를,
 못할거라곤,
 상상도,
 못했네,
 나이스,
 별,
 재미도없는거,
 우려먹어,
  ,
 챔프에서,
 방송,
 몇번했더라,
  ,
 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,
 일의,
 금요일,
 나이트메어시리즈와,
 함께,
 가장,
 많은,
 시리즈를,
 양산해냈던,
 